# How-To Guide

The following how-to guides illustrate specific tasks that can be performed with PyScat. They assume some familiarity with the package and its basic concepts. For introductory tutorials, please refer to the [Getting Started](getting_started.ipynb) section.

## How to create a parameter ensemble from an optimization run?

Ensemble-based uncertainty quantification can be a relatively cheap way to estimate prediction uncertainty ([ref](https://doi.org/10.1109/TCBB.2022.3213914)).
One way to create a parameter ensemble is to use the parameter sets encountered during an optimization run that meet a certain objective function threshold.

Here is an example of how to create a `pypesto.Ensemble` parameter ensemble from a `SacessOptimizer` optimization run:

In [ ]:
from pyscat import SacessOptimizer, EvalLogger, ThresholdSelector
from pyscat.examples import problem_schwefel
import logging
from scipy.stats import chi2

optimizer = SacessOptimizer(
    num_workers=4,
    max_walltime_s=2,
    sacess_loglevel=logging.WARNING,
)
problem = problem_schwefel

# Let's pretend the objective function is a negative log-likelihood
#  and set a threshold for the 95% confidence region accordingly.
threshold = chi2.ppf(0.95, df=1) / 2
print(f"Using threshold: {threshold:.2f}")
selector = ThresholdSelector(mode="abs", threshold=threshold, dim=problem.dim)

with EvalLogger(selector).attach(problem):
    result = optimizer.minimize(problem)

ensemble = selector.to_ensemble()
print(f"Created ensemble with {ensemble.n_x} members.")

Note that this will record all objective function evaluations during the optimization run within the given threshold. It does not check for admissibility of the parameters. Make sure any local optimizers respect the problem bounds to avoid inadmissible parameter sets in the ensemble.